<h1> Feature Engineering </h1>

In this notebook, you will learn how to incorporate feature engineering into your pipeline.
<ul>
<li> Working with feature columns </li>
<li> Adding feature crosses in TensorFlow </li>
<li> Reading data from BigQuery </li>
<li> Creating datasets using Dataflow </li>
<li> Using a wide-and-deep model </li>
</ul>

Apache Beam only works in Python 2 at the moment, so we're going to switch to the Python 2 kernel. In the above menu, click the dropdown arrow and select `python2`. After that, run the following to ensure we've installed Beam.

In [3]:
%%bash
pip install apache-beam[gcp]==2.9.0

  Using cached https://files.pythonhosted.org/packages/d4/3d/90aa15779e884feebae4b0c26cad6f52cd4040397a94deb58dad9c8b7300/apache_beam-2.9.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/2d/99/b2c4e9d5a30f6471e410a146232b4118e697fa3ffc06d6a65efde84debd0/futures-3.2.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b2/a8/ad407cd2a56a052d92f602e164a9e16bede22079252af0db3838f375b6a8/protobuf-3.8.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dc/83/15f7833b70d3e067ca91467ca245bae0f6fe56ddc7451aa0dc5606b120f2/pytz-2018.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cc/3e/29f92b7aeda5b078c86d14f550bf85cff809042e3429ace7af6193c3bc9f/typing-3.6.6-py2-none-any.whl
  Using cached https://files.pythonhosted

In [2]:
import tensorflow as tf
import apache_beam as beam
import shutil
print(tf.__version__)

1.13.1


<h2> 1. Environment variables for project and bucket </h2>

<li> Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos </li>
<li> Cloud training often involves saving and restoring model files. Therefore, we should <b>create a single-region bucket</b>. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available) </li>
</ol>
<b>Change the cell below</b> to reflect your Project ID and bucket name.


In [25]:
import os
PROJECT = 'cloud-training-demos'    # CHANGE THIS
BUCKET = 'your_regional_bucket' # REPLACE WITH YOUR BUCKET NAME. Use a regional bucket in the region you selected.
REGION = 'us-central1' # Choose an available region for Cloud MLE from https://cloud.google.com/ml-engine/docs/regions.

In [5]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.13' 

## ensure we're using python2 env
os.environ['CLOUDSDK_PYTHON'] = 'python2'

In [6]:
%%bash
## ensure gcloud is up to date
sudo apt-get update && sudo apt-get --only-upgrade install kubectl google-cloud-sdk google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-app-engine-go google-cloud-sdk-cloud-build-local google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python google-cloud-sdk-cbt google-cloud-sdk-bigtable-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-datalab google-cloud-sdk-app-engine-java

gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

Get:2 http://security.debian.org stretch/updates InRelease [94.3 kB]
Get:3 http://packages.cloud.google.com/apt cloud-sdk-stretch InRelease [6,377 B]
Ign:4 http://deb.debian.org/debian stretch InRelease
Get:5 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Hit:6 https://nvidia.github.io/libnvidia-container/debian9/amd64  InRelease
Hit:7 https://nvidia.github.io/nvidia-container-runtime/debian9/amd64  InRelease
Hit:8 https://nvidia.github.io/nvidia-docker/debian9/amd64  InRelease
Hit:9 https://deb.nodesource.com/node_11.x stretch InRelease
Get:10 http://packages.cloud.google.com/apt google-compute-engine-stretch-stable InRelease [3,843 B]
Get:11 http://deb.debian.org/debian stretch-backports InRelease [91.8 kB]
Get:1 https://packages.cloud.google.com/apt kubernetes-xenial InRelease [8,993 B]
Hit:12 http://deb.debian.org/debian stretch Release
Hit:13 http://packages.cloud.google.com/apt google-cloud-packages-archive-keyring-stretch InRelease
Get:14 https://download.docke

dpkg-preconfigure: unable to re-open stdin: No such file or directory


<h2> 2. Specifying query to pull the data </h2>

Let's pull out a few extra columns from the timestamp.

In [7]:
def create_query(phase, EVERY_N):
  if EVERY_N == None:
    EVERY_N = 4 #use full dataset
    
  #select and pre-process fields
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  DAYOFWEEK(pickup_datetime) AS dayofweek,
  HOUR(pickup_datetime) AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """
  
  #add subsampling criteria by modding with hashkey
  if phase == 'train': 
    query = "{} AND ABS(HASH(pickup_datetime)) % {} < 2".format(base_query,EVERY_N)
  elif phase == 'valid': 
    query = "{} AND ABS(HASH(pickup_datetime)) % {} == 2".format(base_query,EVERY_N)
  elif phase == 'test':
    query = "{} AND ABS(HASH(pickup_datetime)) % {} == 3".format(base_query,EVERY_N)
  return query
    
print(create_query('valid', 100)) #example query using 1% of data


SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  DAYOFWEEK(pickup_datetime) AS dayofweek,
  HOUR(pickup_datetime) AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
   AND ABS(HASH(pickup_datetime)) % 100 == 2


Try the query above in https://bigquery.cloud.google.com/table/nyc-tlc:yellow.trips if you want to see what it does (ADD LIMIT 10 to the query!)

<h2> 3. Preprocessing Dataflow job from BigQuery </h2>

This code reads from BigQuery and saves the data as-is on Google Cloud Storage.  We can do additional preprocessing and cleanup inside Dataflow, but then we'll have to remember to repeat that prepreprocessing during inference. It is better to use tf.transform which will do this book-keeping for you, or to do preprocessing within your TensorFlow model. We will look at this in future notebooks. For now, we are simply moving data from BigQuery to CSV using Dataflow.

While we could read from BQ directly from TensorFlow (See: https://www.tensorflow.org/api_docs/python/tf/contrib/cloud/BigQueryReader), it is quite convenient to export to CSV and do the training off CSV.  Let's use Dataflow to do this at scale.

Because we are running this on the Cloud, you should go to the GCP Console (https://console.cloud.google.com/dataflow) to look at the status of the job. It will take several minutes for the preprocessing job to launch.

In [8]:
%%bash
if gsutil ls | grep -q gs://${BUCKET}/taxifare/ch4/taxi_preproc/; then
  gsutil -m rm -rf gs://$BUCKET/taxifare/ch4/taxi_preproc/
fi

First, let's define a function for preprocessing the data

In [9]:
import datetime

####
# Arguments:
#   -rowdict: Dictionary. The beam bigquery reader returns a PCollection in
#     which each row is represented as a python dictionary
# Returns:
#   -rowstring: a comma separated string representation of the record with dayofweek
#     converted from int to string (e.g. 3 --> Tue)
####
def to_csv(rowdict):
  days = ['null', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
  CSV_COLUMNS = 'fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key'.split(',')
  rowdict['dayofweek'] = days[rowdict['dayofweek']]
  rowstring = ','.join([str(rowdict[k]) for k in CSV_COLUMNS])
  return rowstring


####
# Arguments:
#   -EVERY_N: Integer. Sample one out of every N rows from the full dataset.
#     Larger values will yield smaller sample
#   -RUNNER: 'DirectRunner' or 'DataflowRunner'. Specfy to run the pipeline
#     locally or on Google Cloud respectively. 
# Side-effects:
#   -Creates and executes dataflow pipeline. 
#     See https://beam.apache.org/documentation/programming-guide/#creating-a-pipeline
####
def preprocess(EVERY_N, RUNNER):
  job_name = 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  print('Launching Dataflow job {} ... hang on'.format(job_name))
  OUTPUT_DIR = 'gs://{0}/taxifare/ch4/taxi_preproc/'.format(BUCKET)

  #dictionary of pipeline options
  options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
    'project': PROJECT,
    'runner': RUNNER
  }
  #instantiate PipelineOptions object using options dictionary
  opts = beam.pipeline.PipelineOptions(flags=[], **options)
  #instantantiate Pipeline object using PipelineOptions
  with beam.Pipeline(options=opts) as p:
      for phase in ['train', 'valid']:
        query = create_query(phase, EVERY_N) 
        outfile = os.path.join(OUTPUT_DIR, '{}.csv'.format(phase))
        (
          p | 'read_{}'.format(phase) >> beam.io.Read(beam.io.BigQuerySource(query=query))
            | 'tocsv_{}'.format(phase) >> beam.Map(to_csv)
            | 'write_{}'.format(phase) >> beam.io.Write(beam.io.WriteToText(outfile))
        )
  print("Done")

Now, let's run pipeline locally. This takes upto <b>5 minutes</b>.  You will see a message "Done" when it is done.

In [11]:
# This will result in about 40k training rows
preprocess(50000, 'DirectRunner') 

Launching Dataflow job preprocess-taxifeatures-190612-224426 ... hang on


Done


Wait for "Done".

Let's find out how many rows of training data we now have.

In [15]:
%%bash
mkdir preproc
cd preproc
gsutil -m cp gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv* .
wc -l train*

   1000 train.csv-00000-of-00044
   1000 train.csv-00001-of-00044
   1000 train.csv-00002-of-00044
   1000 train.csv-00003-of-00044
   1000 train.csv-00004-of-00044
   1000 train.csv-00005-of-00044
   1000 train.csv-00006-of-00044
   1000 train.csv-00007-of-00044
   1000 train.csv-00008-of-00044
   1000 train.csv-00009-of-00044
   1000 train.csv-00010-of-00044
   1000 train.csv-00011-of-00044
   1000 train.csv-00012-of-00044
   1000 train.csv-00013-of-00044
   1000 train.csv-00014-of-00044
   1000 train.csv-00015-of-00044
    837 train.csv-00016-of-00044
   1000 train.csv-00017-of-00044
   1000 train.csv-00018-of-00044
   1000 train.csv-00019-of-00044
   1000 train.csv-00020-of-00044
   1000 train.csv-00021-of-00044
   1000 train.csv-00022-of-00044
   1000 train.csv-00023-of-00044
   1000 train.csv-00024-of-00044
   1000 train.csv-00025-of-00044
   1000 train.csv-00026-of-00044
   1000 train.csv-00027-of-00044
   1000 train.csv-00028-of-00044
   1000 train.csv-00029-of-00044
   1000 tr

Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00000-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00001-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00002-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00003-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00004-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00005-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00008-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00014-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00006-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00010-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00007-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00011-of-00044...
Copying gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00009-of-00044...
Copying gs:/

## 4. Run Beam pipeline on Cloud Dataflow (optional)

If you're content to run a training job with ~40k rows of training data, skip this entire step and proceed to Train on Cloud.

For better results, run the steps below to start a Dataflow job in the cloud to create about 200k rows. This will take 10-15 minutes.

In [24]:
%%bash
if gsutil ls | grep -q gs://${BUCKET}/taxifare/ch4/taxi_preproc/; then
  gsutil -m rm -rf gs://$BUCKET/taxifare/ch4/taxi_preproc/
fi

In [17]:
preprocess(100*100, 'DataflowRunner')
# A divisor of 10,000 will result in about 200k training rows
# Change first arg to None to preprocess full dataset

Launching Dataflow job preprocess-taxifeatures-190612-225424 ... hang on
Done


Monitor job progress on the [Cloud Console, in the Dataflow](https://console.cloud.google.com/dataflow) section.

Once the job completes, observe the files created in Google Cloud Storage.

In [20]:
%%bash
gsutil ls -l gs://$BUCKET/taxifare/ch4/taxi_preproc/

  48668142  2019-06-12T23:04:17Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00000-of-00001
    121757  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00000-of-00044
    113309  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00001-of-00044
    112447  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00002-of-00044
    113096  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00003-of-00044
    115121  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00004-of-00044
    113680  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00005-of-00044
    113681  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00006-of-00044
    113421  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00007-of-00044
    113543  2019-06-12T22:47:03Z  gs://drfib-ml/taxifare/ch4/taxi_preproc/train.csv-00008-of-00044
    115091

In [22]:
%%bash
#print first 10 lines of first shard of train.csv
gsutil cat "gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv-00000-of-*" | head

11.3,Fri,13,-73.975687,40.76003,-73.999972,40.762227,2.0,2009-09-11 13:56:00.000000-73.975740.7640.7622-74
6.1,Wed,12,-73.979987,40.757182,-73.999998,40.748792,2.0,2009-12-16 12:26:39.000000-73.9840.757240.7488-74
21.3,Tue,17,-74.00001,40.72167,-73.933223,40.679975,2.0,2012-05-15 17:38:00.000000-7440.721740.68-73.9332
6.5,Sat,15,-73.992025,40.725997,-74.000012,40.72185,2.0,2012-08-18 15:13:00.000000-73.99240.72640.7219-74
7.0,Sun,21,-73.991645,40.74996,-74.000044,40.730599,2.0,2014-04-06 21:49:00.000000-73.991640.7540.7306-74
14.0,Tue,22,-73.9999771118,40.7216110229,-73.9599914551,40.7626457214,2.0,2015-06-09 22:19:48.000000-7440.721640.7626-73.96
10.9,Tue,14,-73.999958,40.730595,-73.980769,40.763996,2.0,2009-01-13 14:35:50.000000-7440.730640.764-73.9808
10.9,Sat,18,-73.999975,40.717953,-73.99127,40.750233,2.0,2009-04-18 18:31:00.000000-7440.71840.7502-73.9913
5.7,Thu,1,-73.99998,40.738628,-73.981518,40.741005,2.0,2009-09-10 01:02:14.000000-7440.738640.741-73.9815
6.1,Sat,1,-73.999968,

## 5. Develop model with new inputs

Download the first shard of the preprocessed data to enable local development.

In [ ]:
%%bash
if [ -d sample ]; then
  rm -rf sample
fi
mkdir sample
gsutil cat "gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv-00000-of-*" > sample/train.csv
gsutil cat "gs://$BUCKET/taxifare/ch4/taxi_preproc/valid.csv-00000-of-*" > sample/valid.csv

We have two new inputs in the INPUT_COLUMNS, three engineered features, and the estimator involves bucketization and feature crosses.

In [ ]:
%%bash
grep -A 20 "INPUT_COLUMNS =" taxifare/trainer/model.py

In [ ]:
%%bash
grep -A 50 "build_estimator" taxifare/trainer/model.py

In [ ]:
%%bash
grep -A 15 "add_engineered(" taxifare/trainer/model.py

Try out the new model on the local sample (this takes <b>5 minutes</b>) to make sure it works fine.

In [ ]:
%%bash
rm -rf taxifare.tar.gz taxi_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
python -m trainer.task \
  --train_data_paths=${PWD}/sample/train.csv \
  --eval_data_paths=${PWD}/sample/valid.csv  \
  --output_dir=${PWD}/taxi_trained \
  --train_steps=10 \
  --job-dir=/tmp

In [ ]:
%%bash
ls taxi_trained/export/exporter/

You can use ```saved_model_cli``` to look at the exported signature. Note that the model doesn't need any of the engineered features as inputs. It will compute latdiff, londiff, euclidean from the provided inputs, thanks to the ```add_engineered``` call in the serving_input_fn.

In [ ]:
%%bash
model_dir=$(ls ${PWD}/taxi_trained/export/exporter | tail -1)
saved_model_cli show --dir ${PWD}/taxi_trained/export/exporter/${model_dir} --all

In [ ]:
%%writefile /tmp/test.json
{"dayofweek": "Sun", "hourofday": 17, "pickuplon": -73.885262, "pickuplat": 40.773008, "dropofflon": -73.987232, "dropofflat": 40.732403, "passengers": 2}

In [ ]:
%%bash
model_dir=$(ls ${PWD}/taxi_trained/export/exporter)
gcloud ml-engine local predict \
  --model-dir=${PWD}/taxi_trained/export/exporter/${model_dir} \
  --json-instances=/tmp/test.json

## 6. Train on cloud

This will take <b> 10-15 minutes </b> even though the prompt immediately returns after the job is submitted. If you did part 4 to generate a larger data set, multiply x5 the train_steps below. Your job will take about 30 minutes in this case. If you have enough quota, you could use the PREMIUM_1 scale tier to go about 4x faster.


In [23]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/ch4/taxi_trained
JOBNAME=lab4a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=${PWD}/taxifare/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=$TFVERSION \
  -- \
  --train_data_paths="gs://$BUCKET/taxifare/ch4/taxi_preproc/train*" \
  --eval_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/valid*"  \
  --train_batch_size=128 --nbuckets=21 --hidden_units="144 89 55" \
  --train_steps=46368 \
  --output_dir=$OUTDIR

gs://drfib-ml/taxifare/ch4/taxi_trained us-central1 lab4a_190612_230832
jobId: lab4a_190612_230832
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [lab4a_190612_230832] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe lab4a_190612_230832

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs lab4a_190612_230832


Monitor the job in the GCP Console > AI Platform > Jobs.

<b>Do NOT proceed until the job is done.</b>

To view progress in Tensorboard, note the OUTDIR above and run Tensorboard in Cloud Shell:

> tensorboard --logdir=$OUTDIR --port=8080

Connect with Web Preview as before.

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/taxifare/ch4/taxi_trained/export/exporter | tail -1

In [ ]:
%%bash
model_dir=$(gsutil ls gs://${BUCKET}/taxifare/ch4/taxi_trained/export/exporter | tail -1)
saved_model_cli show --dir ${model_dir} --all

In [ ]:
%%bash
model_dir=$(gsutil ls gs://${BUCKET}/taxifare/ch4/taxi_trained/export/exporter | tail -1)
gcloud ml-engine local predict \
  --model-dir=${model_dir} \
  --json-instances=/tmp/test.json

### Optional: deploy model to cloud

In [ ]:
%%bash
MODEL_NAME="feateng"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/taxifare/ch4/taxi_trained/export/exporter | tail -1)
echo "Run these commands one-by-one (the very first time, you'll create a model and then create a version)"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

In [ ]:
%%bash
gcloud ml-engine predict --model=feateng --version=v1 --json-instances=/tmp/test.json

<h2> 7. Hyper-parameter tune </h2>

Look at <a href="hyperparam.ipynb">hyper-parameter tuning notebook</a> to decide what parameters to use for model. Based on that run, I ended up choosing:
<ol>
<li> train_batch_size: 128 </li>
<li> nbuckets: 21 </li>
<li> hidden_units: "144 89 55" </li>   
<li> nSteps: 317511</li>
</ol>

This gives an RMSE of 5, a considerable improvement from the 8.3 we were getting earlier ... Let's try this over a larger dataset.

# Optional: Run Cloud training on 2 million row dataset

This run uses as input 2 million rows and takes ~30 minutes with 19 workers (PREMIUM_1 pricing tier). The model is exactly the same as above. The only changes are to the input (to use the larger dataset) and to the scale tier. Because the Dataflow preprocessing takes about 15 minutes, we train here using CSV files in a public bucket.

In [ ]:
%%bash

OUTDIR=gs://${BUCKET}/taxifare/feateng2m
JOBNAME=lab4a_$(date -u +%y%m%d_%H%M%S)
TIER=STANDARD_1 
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=$TIER \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://cloud-training-demos/taxifare/ch4/taxi_preproc/train*" \
   --eval_data_paths="gs://cloud-training-demos/taxifare/ch4/taxi_preproc/valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=2178309 \
   --train_batch_size=128 --nbuckets=21 --hidden_units="144 89 55"

### Start Tensorboard

In [ ]:
from google.datalab.ml import TensorBoard
OUTDIR='gs://{0}/taxifare/feateng2m'.format(BUCKET)
print(OUTDIR)
TensorBoard().start(OUTDIR)

### Stop Tensorboard

In [ ]:
pids_df = TensorBoard.list()
if not pids_df.empty:
    for pid in pids_df['pid']:
        TensorBoard().stop(pid)
        print('Stopped TensorBoard with pid {}'.format(pid))

The RMSE after training on the 2-million-row dataset is ~\$4.  This graph shows the improvements so far ...

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame({'Lab' : pd.Series(['1a', '2-3', '4a', '4b', '4c']),
              'Method' : pd.Series(['Heuristic Benchmark', 'tf.learn', '+Feature Eng.', '+ Hyperparam', '+ 2m rows']),
              'RMSE': pd.Series([8.026, 9.4, 8.3, 5.0, 3.03]) })

ax = sns.barplot(data = df, x = 'Method', y = 'RMSE')
ax.set_ylabel('RMSE (dollars)')
ax.set_xlabel('Labs/Methods')
plt.plot(np.linspace(-20, 120, 1000), [5] * 1000, 'b');

In [ ]:
%%bash
gsutil -m mv gs://${BUCKET}/taxifare/ch4/ gs://${BUCKET}/taxifare/ch4_1m/

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License